In [ ]:
# DEPRECATED

# !gcloud auth login --enable-gdrive-access --update-adc

In [ ]:
!pip install pyspark
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 44.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a4313d171d2cebc4553143a332463ee5361b215911f61da6e16fbd4ff65e573c
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.2 MB/s 


In [ ]:
# We must put into functions_v1.py
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
from google.auth import default
from gspread.models import Worksheet ,Cell
import xlsxwriter
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import row_number, monotonically_increasing_id, desc, concat_ws
from pyspark.sql.functions import col, when, lit, regexp_replace, from_json, max as max_, expr, first
from pyspark.sql.window import Window
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import *
from typing import List
from google.oauth2.sts import Client
import pyspark.sql.functions as F
from xlsxwriter.utility import *
from datetime import date
from datetime import datetime
from datetime import timedelta 
import functools
from functools import reduce
from pyspark.sql.functions import when
from xlsxwriter.utility import xl_col_to_name
from google.auth.impersonated_credentials import Credentials

In [ ]:
!cp "/functions_v1.py" /content

In [ ]:
from functions_v1 import create_dataframe, map_sheet

In [ ]:
first_position = 0
today: date = date.today()
batch_report: str = '1hZARIq4JfaHqo70E3KS8RkMpD9jA0G1VHzQYPp8ugxw'
incis_report: str = '1ZziJpKBwQ1DtHP-JjZOhdCqdnp4ZDBwoihNZYgyInMs'
sheet_incis: str = 'Agosto - 2022'
sheet_month: str = 'Agosto 2022'
sheet_diary_work: str = 'Trabajo Diario-Agosto'

In [ ]:
def create_gspread():
  auth.authenticate_user()
  gauth: GoogleAuth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive: GoogleDrive = GoogleDrive(gauth)
  creds, _ = default()
  return gspread.authorize(creds)

In [ ]:
def run_batch_dataFrame(batch_id: str, spark: SparkSession, gc:Client) -> DataFrame :
  worksheet_batch: Worksheet = gc.open_by_key(batch_id).worksheet(sheet_month)
  batch_df: DataFrame = create_dataframe(worksheet_batch,spark)

  return batch_df.withColumn('index_batch', row_number().over(Window.orderBy(monotonically_increasing_id())))\
  .filter(batch_df["Desplanificado"]==False), worksheet_batch

In [ ]:
def run_incis_dataFrame(incis_id: str, spark: SparkSession, gc:Client) -> DataFrame :
  worksheet_incis: Worksheet = gc.open_by_key(incis_id).worksheet(sheet_incis)
  incis_df: DataFrame = create_dataframe(worksheet_incis,spark)
  incis_df = incis_df.filter(~incis_df["Job Control M"].isin("Job Control M"))
  incis_df = incis_df.select("Job Control M"," + Info Error / Acción realizada")
  return incis_df.withColumn('index_incis', row_number().over(Window.orderBy(monotonically_increasing_id()))), worksheet_incis

In [ ]:
def run_diarywork_dataFrame(diary_work_id: str, spark: SparkSession, gc:Client) -> DataFrame :
  worksheet_diary_work: Worksheet = gc.open_by_key(diary_work_id).worksheet(sheet_diary_work)
  diary_work_df: DataFrame = create_dataframe(worksheet_diary_work,spark)
  diary_work_df = diary_work_df.filter(~diary_work_df["Job en Control-M"].isin("Job en Control-M"))
  diary_work_df =  diary_work_df.filter(diary_work_df["Estado"]=="KO")
  return diary_work_df, worksheet_diary_work

In [ ]:
def join_dataFrames(df1:DataFrame, df2:DataFrame) -> DataFrame :
  return df1.join(df2,(df1["Job en Control-M"]==df2["Job Control M"]),"inner")\
                                     .drop("Job Control M")

In [ ]:
def join_dataFrames3(df1:DataFrame, df2:DataFrame) -> DataFrame :
  return df1.join(df2,(df1[" + Info Error / Acción realizada"]==df2[" + Info Error / Acción realizada"]),"inner")\
                                     .drop(df1[" + Info Error / Acción realizada"])

In [ ]:
def filter_batch_dataFrame(df:DataFrame, first_position: int, diary_work_df: DataFrame):
  first_position = diary_work_df.count()
  print(first_position)

  df: DataFrame = df.withColumn('new_index', row_number().over(Window.orderBy(monotonically_increasing_id()))+first_position+2)
  
  return df

In [ ]:
def map_df_in_diary_worksheet(joined_index_df: DataFrame, worksheet_diary_batch: Worksheet, colu: str) -> None:
  try:    
    colum: str = xlsxwriter.utility.xl_col_to_name(worksheet_diary_batch.get_all_values()[0].index(colu))
    listIndex: List[tuple] = joined_index_df.select("new_index",str(colu)).where(col(colu).isNotNull()).rdd.map(tuple).collect()
    range: str = colum+"2:"+colum+"10000"
    cell_list: list[Cell] = worksheet_diary_batch.range(range)
    print(cell_list)
    i=0
    for cell in (cell for cell in cell_list if cell.row in list(map(lambda x:x[0],listIndex))):
      cell.value = listIndex[i][1]
      i=i+1
    worksheet_diary_batch.update_cells(cell_list)
  except:
    print(colum+" no esta en esta pagina")

In [ ]:
def map_diary_work_in_sheet(column_list: list, df:DataFrame, worksheet_to_map: Worksheet) -> None :
  for column_l in column_list:
    map_df_in_diary_worksheet(df,worksheet_to_map,column_l)

In [ ]:
def run_incis(incis_report:str, batch_report:str) -> None :
  gc: Client = create_gspread()
  spark: SparkSession = SparkSession.builder.getOrCreate()

  batch_df , batch_sheet = run_batch_dataFrame(batch_report,spark,gc)
  incis_df, incis_sheet = run_incis_dataFrame(incis_report,spark,gc)
  diary_work_df, diary_work_sheet = run_diarywork_dataFrame(batch_report,spark,gc)
  joined_jobs_df: DataFrame  = join_dataFrames(diary_work_df,incis_df).drop(diary_work_df[" + Info Error / Acción realizada"]).dropDuplicates()
  droped_jobs_df = joined_jobs_df.dropDuplicates(['index_incis'])
  worksheet_diary_work: Worksheet = gc.open_by_key(batch_report).worksheet(sheet_diary_work)
  column_incis_work: list = joined_jobs_df[" + Info Error / Acción realizada"]
  columns_diary_work: list = diary_work_df.columns
  index_df: DataFrame = filter_batch_dataFrame(droped_jobs_df, first_position, diary_work_df).drop('index_incis')
  index_df.show()
  map_diary_work_in_sheet(columns_diary_work, index_df, worksheet_diary_work) 


In [ ]:
run_incis(incis_report,batch_report)

15
+--------------------+-----+-----------------------+--------------------+------+------------+--------+------+----------+--------+--------------------------------+---------+
|    Job en Control-M|Malla|Job en Argos - Dataproc|               Notas|Acción|Current Date|   ODATE|Estado|Comentario|Revisado| + Info Error / Acción realizada|new_index|
+--------------------+-----+-----------------------+--------------------+------+------------+--------+------+----------+--------+--------------------------------+---------+
|XDTI_D_XCSF_CDDCO...|     |   xcsf-gl-krb-inr-c...|   Si no llega mirar|      |  2022-08-04|20220802|    KO|          |        |                      Se relanza|       18|
|XDTI_D_XCSF_CDDCO...|     |   xcsf-gl-krb-inr-c...|   Si no llega mirar|      |  2022-08-04|20220802|    KO|          |        |                      Se relanza|       19|
|XDTI_D_XCSF_CDDCO...|     |   xcsf-gl-krb-inr-c...|   Si no llega mirar|      |  2022-08-04|20220802|    KO|          |        |   